In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras

from sklearn.model_selection import StratifiedKFold
from keras import layers, Sequential, Input

from sklearn import preprocessing

In [2]:
#Reset column names for training set
train_columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level']
test_columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level']

In [3]:
train_data = pd.read_csv(r"data\KDDTrain+.txt")
test_data = pd.read_csv(r"data\KDDTest+.txt")
train_data.columns = train_columns
test_data.columns = test_columns

In [4]:
train_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,subclass,difficulty_level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [5]:
test_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,subclass,difficulty_level
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,15
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,11
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,21


In [6]:
train_data = train_data.drop(columns=["difficulty_level"])
test_data = test_data.drop(columns=["difficulty_level"])

In [7]:
print(train_data.isnull().values.sum())
print(test_data.isnull().values.sum())

0
0


In [8]:
def preprocess(data, scaler, encoder, train):
    features = data.copy()

    numerical_columns = features.select_dtypes(include=["number"]).columns.tolist()
    categorical_columns = features.select_dtypes(include=["object"]).columns.tolist()

    if train:
        scaled = scaler.fit_transform(features[numerical_columns])
        encoded = encoder.fit_transform(features[categorical_columns]).toarray()
        dummy_columns = encoder.get_feature_names_out(categorical_columns)
    else:
        scaled = scaler.transform(features[numerical_columns])
        encoded = encoder.transform(features[categorical_columns]).toarray()
        dummy_columns = encoder.get_feature_names_out(categorical_columns)

    scaled_data = pd.DataFrame(scaled, columns=numerical_columns, index=features.index)

    encoded_data = pd.DataFrame(encoded, columns=dummy_columns, index=features.index)

    features = pd.concat([scaled_data, encoded_data], axis=1)
    return features, encoder

In [9]:
all_data = pd.concat([train_data, test_data])

In [10]:
tmp = all_data.pop("subclass")

In [11]:
encoder = preprocessing.OneHotEncoder(handle_unknown="ignore", sparse_output=True)
scaler = preprocessing.MinMaxScaler()
all_data, encoder = preprocess(all_data, scaler, encoder, True)

In [12]:
all_data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0.0,5.753774e-07,2.542106e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22539,0.0,2.297162e-07,7.160648e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22540,0.0,3.952277e-05,6.346868e-06,0.0,0.0,0.0,0.019802,0.0,1.0,0.000134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22541,0.0,3.043558e-08,3.206260e-08,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
classlist = []
check1 = (
    "apache2",
    "back",
    "land",
    "neptune",
    "mailbomb",
    "pod",
    "processtable",
    "smurf",
    "teardrop",
    "udpstorm",
    "worm",
)
check2 = ("ipsweep", "mscan", "nmap", "portsweep", "saint", "satan")
check3 = (
    "buffer_overflow",
    "loadmodule",
    "perl",
    "ps",
    "rootkit",
    "sqlattack",
    "xterm",
)
check4 = (
    "ftp_write",
    "guess_passwd",
    "httptunnel",
    "imap",
    "multihop",
    "named",
    "phf",
    "sendmail",
    "Snmpgetattack",
    "spy",
    "snmpguess",
    "warezclient",
    "warezmaster",
    "xlock",
    "xsnoop",
)

DoSCount = 0
ProbeCount = 0
U2RCount = 0
R2LCount = 0
NormalCount = 0

for item in tmp:
    if item in check1:
        classlist.append("DoS")
        DoSCount = DoSCount + 1
    elif item in check2:
        classlist.append("Probe")
        ProbeCount = ProbeCount + 1
    elif item in check3:
        classlist.append("U2R")
        U2RCount = U2RCount + 1
    elif item in check4:
        classlist.append("R2L")
        R2LCount = R2LCount + 1
    else:
        classlist.append("Normal")
        NormalCount = NormalCount + 1

In [14]:
all_data["class"] = classlist
all_data["class"].value_counts()

class
Normal    77231
DoS       53386
Probe     14077
R2L        3702
U2R         119
Name: count, dtype: int64

In [15]:
all_data.isnull().values.sum()

0

In [16]:
all_data.to_csv(r"data\cnn1d_bilstm.csv", index=False)